In [184]:
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

import kernel

In [185]:
def eye(N):
    return tf.diag(tf.ones(tf.stack([N, ]), dtype='float32'))

In [186]:
# Generate synthetic data.
N_all = 200
N = 5
inputs = 5 * np.linspace(0, 1, num=N_all)[:, np.newaxis]
outputs = np.sin(inputs)

# selects training and test
idx = np.arange(N_all)
np.random.shuffle(idx)
xtrain = inputs[idx[:N]]
ytrain = outputs[idx[:N]]

In [187]:
inducing_inputs = xtrain
points1 = inducing_inputs
points2 = points1

In [219]:
points1 = np.array([[2.06030151], [2.33668342],[2.21105528],[1.75879397], [2.91457286]])
points2 = np.array([[2.06030151], [2.33668342],[2.21105528],[1.75879397], [2.91457286]])
points1

array([[ 2.06030151],
       [ 2.33668342],
       [ 2.21105528],
       [ 1.75879397],
       [ 2.91457286]])

In [189]:
input_dim = 1
lengthscale = 1.0
std_dev=1.0
white=0.5
input_scaling=False
MAX_DIST = 1e8

In [190]:
if input_scaling:
    lengthscale = tf.Variable(lengthscale * tf.ones([input_dim]))
else:
    lengthscale = tf.Variable([lengthscale], dtype=tf.float32)

std_dev = tf.Variable([std_dev], dtype=tf.float32)

In [191]:
if all(points2) == all(points1):
    white_noise = white * eye(tf.shape(points1)[0]) 
else: 
    white_noise = 0.0  

In [192]:
#white_noise
#lengthscale

In [193]:
#white_noise

In [194]:
#lengthscale = tf.cast(lengthscale, tf.float64)
#lengthscale

In [195]:
points1 = points1 / lengthscale * lengthscale
#points1

In [196]:
points2 = points2 / lengthscale * lengthscale

In [197]:
#points2

In [198]:
#tf.reduce_sum(points1 ** 2, 1)

In [199]:
magnitude_square1 = tf.expand_dims(tf.reduce_sum(points1 ** 2, 1), 1)
magnitude_square2 = tf.expand_dims(tf.reduce_sum(points2 ** 2, 1), 1)
#magnitude_square2

In [200]:
#doppio_prodotto = - 2 * tf.matmul(points1, tf.transpose(points2))

In [201]:
#distances_prima_parte = magnitude_square1 + doppio_prodotto

In [202]:
distances = (magnitude_square1 
             - 2 * tf.matmul(points1, tf.transpose(points2)) +
            tf.transpose(magnitude_square2))
#distances

In [203]:
distances_root = tf.sqrt(distances)/lengthscale
#distances_root

In [204]:
distances = tf.clip_by_value(distances, 0.0, MAX_DIST)

In [205]:
#distances_root

In [206]:
constant = tf.sqrt(3.0)

In [207]:
first_term=(constant*distances_root + 1)*std_dev
#first_term

In [208]:
second_term = tf.exp(-constant*distances_root)
#second_term

In [209]:
kernel_matrix = tf.multiply(first_term,second_term)

In [210]:
#kernel_matrix

In [211]:
kernel_nondiag = kernel_matrix + white_noise

In [212]:
#kernel_nondiag

In [213]:
kernel_diag = ((std_dev ** 2) + white) * tf.ones([tf.shape(points1)[0]])

In [214]:
#kernel_diag

In [215]:
kernel_chol = tf.cholesky(kernel_nondiag)

In [216]:
chol_inverse = tf.matrix_inverse(kernel_chol)

In [217]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [222]:
#print("questo è il primo punto")
#print points1.eval(session=sess)
#print("questo è il vettore al quadrato, magnitude_square1")
#print magnitude_square1.eval(session=sess)
#print magnitude_square2.eval(session=sess)
#print("questo è il doppio prodotto")
#print doppio_prodotto.eval(session=sess)
#print("prima parte distanza")
#print distances_prima_parte.eval(session=sess)
#print("questa è la distanza al quadrato")
#print distances.eval(session=sess)
#print("questa è la square root")
#print distances_root.eval(session=sess)
#print("è il primo termine")
#print first_term.eval(session=sess)
#print("questo è il secondo termine")
#print second_term.eval(session=sess)
print("kernel")
print kernel_nondiag.eval(session=sess)
print("cholesky")
print kernel_chol.eval(session=sess)
print kernel_chol
print("inverse of cholesky")
print chol_inverse.eval(session=sess)
print chol_inverse

kernel
[[ 1.5         0.91618353  0.97130114  0.90298194  0.56466204]
 [ 0.91618353  1.5         0.97949588  0.73541546  0.73541504]
 [ 0.97130114  0.97949588  1.5         0.81476802  0.65594113]
 [ 0.90298194  0.73541546  0.81476802  1.5         0.40550056]
 [ 0.56466204  0.73541504  0.65594113  0.40550056  1.5       ]]
cholesky
[[ 1.22474492  0.          0.          0.          0.        ]
 [ 0.7480607   0.96974492  0.          0.          0.        ]
 [ 0.79306406  0.39828601  0.84404838  0.          0.        ]
 [ 0.73728162  0.18962102  0.18308558  0.94177455  0.        ]
 [ 0.46104461  0.40270972  0.15391247 -0.04136929  1.04874325]]
Tensor("Cholesky_5:0", shape=(5, 5), dtype=float32)
inverse of cholesky
[[ 0.81649655  0.          0.          0.          0.        ]
 [-0.62984502  1.03119898  0.          0.          0.        ]
 [-0.46996784 -0.48659787  1.18476617  0.          0.        ]
 [-0.42102614 -0.11302911 -0.23032436  1.06182528  0.        ]
 [-0.06472533 -0.32901886 -0

In [183]:
#kernel_nondiag